Aluno: André Bamberg Pan
RA:2079844

Importando as bibliotecas

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split

Dataset iris

In [ ]:
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target


DataFrame com as colunas especificadas

In [ ]:
iris1 = iris_df[['petal length (cm)', 'petal width (cm)', 'target']]

Preparação dos dados

In [ ]:
X = iris1[['petal length (cm)', 'petal width (cm)']]
y = iris1.target

Divisão dos Dados em Conjuntos de Treinamento e Teste

In [ ]:
# Mantendo os índices originais ao dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)


Implementação do KNN

In [ ]:
class KNN:
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            self.X = X.values
        else:
            self.X = X
        if isinstance(y, pd.Series):
            self.y = y.values
        else:
            self.y = y

    def predict(self, X_test):
        if isinstance(X_test, pd.DataFrame):
            X_test = X_test.values
        predictions = []
        for x_test in X_test:
            distances = []
            for x_train in self.X:
                distance = np.sqrt(np.sum((x_test - x_train) ** 2))
                distances.append(distance)
            sorted_indices = np.argsort(distances)
            k_nearest_indices = sorted_indices[:self.n_neighbors]
            k_nearest_labels = self.y[k_nearest_indices]
            prediction = np.argmax(np.bincount(k_nearest_labels))
            predictions.append(prediction)
        return predictions


Classificador KNN

In [ ]:
clf = KNN(n_neighbors=3)

Treinamento do Modelo

In [ ]:
clf.fit(X_train, y_train)

Realização de Previsões com os Dados de Teste

In [ ]:
# Chamando o método predict com os dados de teste
y_pred = clf.predict(X_test.values)

print(y_pred)

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 1, 2, 1, 2]


Matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[19,  0,  0],
       [ 0, 15,  0],
       [ 0,  1, 15]])

Teste de acurácia

In [ ]:
num_correct_predictions = (y_pred == y_test).sum()
accuracy = (num_correct_predictions / y_test.shape[0]) * 100
print('Test set accuracy: %.2f%%' % accuracy)

Test set accuracy: 98.00%


Visualizando todos os dados de treino, teste e previsões

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
ax.scatter(x=X_train['petal width (cm)'],
           y=X_train['petal length (cm)'],
           c=y_train,
           cmap='viridis')

ax.scatter(x=X_test['petal width (cm)'],
           y=X_test['petal length (cm)'],
           c=y_test,
           cmap='Accent')

ax.scatter(x=X_test['petal width (cm)'],
           y=X_test['petal length (cm)'],
           c=y_pred, alpha=0.2,
           cmap='Accent')

ax.set(xlim=(0.9,2.6), xticks=[1,1.5,2,2.5],
       ylim=(3,7), yticks=[3,4,5,6,7])

plt.show()

Classificação de 5 novos objetos

In [ ]:
novos_objetos = [
    [4.0, 1.7],  # Novo objeto 1
    [1.1, 3.4],  # Novo objeto 2
    [6.6, 2.2],  # Novo objeto 3
    [5.1, 1.5],  # Novo objeto 4
    [4.3, 2.7]   # Novo objeto 5
]

# Convertendo para um array numpy
novos_objetos_array = np.array(novos_objetos)

# Chamando o método predict para classificar os novos objetos
y_pred_novos_objetos = clf.predict(novos_objetos_array)

print("Classificação dos novos objetos:")
for i, pred in enumerate(y_pred_novos_objetos):
    print(f"Novo objeto {i+1}: Classe {pred}")


Classificação dos novos objetos:
Novo objeto 1: Classe 1
Novo objeto 2: Classe 0
Novo objeto 3: Classe 2
Novo objeto 4: Classe 1
Novo objeto 5: Classe 2


KNN com Ball-tree

In [ ]:
from sklearn.neighbors import BallTree

class KNNBallTree:
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.tree = BallTree(X)
        self.y = np.concatenate((y_train, y_test))

    def predict(self, X_test):
        _, indices = self.tree.query(X_test, k=self.n_neighbors)
        predictions = []
        for idx in indices:
            k_nearest_labels = self.y[idx]
            prediction = np.argmax(np.bincount(k_nearest_labels))
            predictions.append(prediction)
        return predictions


Comparando tempos

In [ ]:
import time

# Medindo o tempo de execução do KNN
start_time = time.time()
clf = KNN(n_neighbors=3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test.values)
end_time = time.time()
execution_time_knn = end_time - start_time

# Medindo o tempo de execução do KNNBallTree
start_time = time.time()
clf_ball_tree = KNNBallTree(n_neighbors=3)
clf_ball_tree.fit(X_train, y_train)
y_pred_ball_tree = clf_ball_tree.predict(X_test.values)
end_time = time.time()
execution_time_ball_tree = end_time - start_time

# Imprimindo os tempos de execução
print("Tempo de execução do KNN:", execution_time_knn)
print("Tempo de execução do KNNBallTree:", execution_time_ball_tree)

Tempo de execução do KNN: 0.06737732887268066
Tempo de execução do KNNBallTree: 0.003215312957763672
